# Segundo Trabalho - PSO (Ritika)

- [1. Implementação](#1.-Implementação)
    - [1.1. Bibliotecas utilizadas](#1.1.-Bibliotecas-utilizadas)
    - [1.2. Classe Partícula](#1.2.-Classe-Partícula)
    - [1.3. Classe PSO](#1.3.-Classe-PSO)
- [2. Execução](#2.-Execução)
    - [2.1. Inicialização](#2.1.-Inicialização)
    - [2.2. Cálculo da velocidade e posição](#2.2.-Cálculo-da-velocidade-e-posição)
    - [2.3. Iterações Restantes](#2.3.-Iterações-Restantes)

In [14]:
from IPython.display import IFrame

IFrame("https://www.youtube.com/embed/HmDjfL3R39M", width=851, height=480)

## 1. Implementação

### 1.1. Bibliotecas utilizadas

In [1]:
import sys
from random import random
from random import seed
import pandas as pd
import copy

### 1.2. Classe Partícula

In [2]:
class Particle:
    def __init__(self, n, f, w):
        # upper and lower bounds
        ub = [0, 0, 0]
        lb = [10, 10, 10]
        
        self.n = n                         # number of variables
        self.x = [round(lb[j] + random() * (ub[j]-lb[j])) for j in range(n)] # current position
        self.pbest = copy.deepcopy(self.x) # best position
        self.f = f                         # objective funct
        self.w = w                         # inertia weight
        self.y = self.f(*self.x)           # fitness value
        self.v = [x*0.1 for x in self.x]   # velocity
        
    def calculate_x(self, v):
        new_x = []
        for i in range(len(self.x)):
            new_x.append(self.x[i] + v[i])
        self.x = new_x
        self.y = self.f(*self.x)
    
    def calculate_v(self, w, c, gbest):
        new_v = []
        for i in range(len(self.v)):
            new_v.append(w["max"]*self.v[i] + c[0]*random()*(self.pbest[i]-self.x[i]) + c[1]*random()*(gbest.x[i]-self.x[i]))
        self.v = new_v

### 1.3. Classe PSO

In [3]:
class PSO:
    def __init__(self, funct, n_var, n_pop, wmax, wmin, c, maxis):
        self.funct = funct # objective funct
        self.n_var = n_var # number of variables
        self.n_pop = n_pop # population size
        self.w     = {     # inertia weight
            "max": wmax,
            "min": wmin
        }
        self.c     = c     # acceleration factor
        self.maxis = maxis # maximum iteration size
        self.popul = []    # population
        self.gbest = None  # global best particle
        
    def initialize(self):
        self.popul = [Particle(self.n_var, self.funct, self.w) for _ in range(self.n_pop)]
        min_y = min([particle.y for particle in self.popul])
        for particle in self.popul:
            if particle.y == min_y:
                self.gbest = particle
                
    def evaluate_pbest(self, particle):
        if self.funct(*particle.x) < self.funct(*particle.pbest):
            particle.pbest = copy.deepcopy(particle.x)
    
    def evaluate_gbest(self):
        # calculate fitness for each particle
        # choose particle with best fitness value as gbest
        for particle in self.popul:
            if particle.y < self.gbest.y:
                self.gbest = copy.deepcopy(particle)
                
    def first_movement(self):
        # calculate velocity and position for each particle
        for particle in self.popul:
            particle.calculate_x(particle.v)
            # select pbest for current particle
            self.evaluate_pbest(particle)
        self.evaluate_gbest()

    def update(self):
        # calculate velocity and position for each particle
        for particle in self.popul:
            particle.calculate_v(self.w, self.c, self.gbest)
            particle.calculate_x(particle.v)
            # select pbest for current particle
            self.evaluate_pbest(particle)
        self.evaluate_gbest()
    
    def run(self):
        for i in range(self.maxis-1):
            self.update()
    
    def display_population(self):
        dict_popul = dict((x,[]) for x in [f"x{i}" for i in range(self.n_var)])
        dict_popul["y"] = []
        for particle in self.popul:
            for i in range(self.n_var):
                dict_popul[f"x{i}"].append(particle.x[i])
            dict_popul["y"].append(particle.y)
        return pd.DataFrame(dict_popul)
    
    def display_gbest(self):
        dict_gbest = dict((x,[]) for x in [f"x{i}" for i in range(self.n_var)])
        for i in range(self.n_var):
            dict_gbest[f"x{i}"].append(self.gbest.x[i])
        dict_gbest["y"] = self.gbest.y
        return pd.DataFrame(dict_gbest)

## 2. Execução

### 2.1. Inicialização

In [4]:
# objective funct for this problem
f = lambda x0, x1, x2: 10*(x0-1)**2 + 20*(x1-2)**2 + 30*(x2-3)**2

# instantiate a PSO object
p = PSO(f, 3, 5, .9, .4, [2,2], 50)
p.initialize()
p.display_population()

,x0,x1,x2,y
0,1,6,2,350
1,7,2,6,630
2,9,6,3,960
3,0,1,3,30
4,2,6,6,600


In [5]:
# moving particles for the first time

p.first_movement()
p.display_population()

,x0,x1,x2,y
0,1.1,6.6,2.2,442.5
1,7.7,2.2,6.6,838.5
2,9.9,6.6,3.3,1218.0
3,0.0,1.1,3.3,28.9
4,2.2,6.6,6.6,826.4


In [6]:
# gbest so far

p.display_gbest()

,x0,x1,x2,y
0,0.0,1.1,3.3,28.9


### 2.2. Cálculo da velocidade e posição

In [7]:
# updating velocity and position for each particle

p.update()
p.display_population()

,x0,x1,x2,y
0,-1.044322,-0.991051,3.379642,225.044101
1,2.202915,0.966985,1.887256,72.958401
2,10.217857,6.706685,3.221979,1294.224678
3,0.000000,1.190000,3.570000,32.869000
4,1.818761,0.336480,5.276313,217.497640


In [8]:
p.display_gbest()

,x0,x1,x2,y
0,0.0,1.19,3.57,32.869


### 2.3. Iterações Restantes

In [9]:
p.run()
p.display_population()

,x0,x1,x2,y
0,-0.739686,0.973175,2.748905,53.243955
1,10.606823,-68.763720,7.137277,101586.502701
2,0.858930,-0.908875,1.764423,215.229604
3,1.647890,1.932150,3.162338,5.080297
4,1.619429,1.374823,-7.802479,3512.460547


In [10]:
p.display_gbest()

,x0,x1,x2,y
0,1.058289,1.994492,3.174637,0.949527
